# Pulizia dati: estrai solo lo Yemen

Questa notebook carica il CSV nella cartella `data/`, individua la colonna paese (se presente), filtra le righe contenenti 'Yemen' (case-insensitive) e salva il risultato in `data/yemen_prices.csv`.

In [ ]:
from pathlib import Path
import pandas as pd

# Cerca file CSV nella cartella data/ in modo robusto (gestisce working dir diversi)
def find_data_dir():
    cwd = Path.cwd()
    # controllo diretto in CWD
    if (cwd / 'data').is_dir():
        return (cwd / 'data').resolve()
    # risalgo i parent
    for parent in cwd.parents:
        if (parent / 'data').is_dir():
            return (parent / 'data').resolve()
    # cerca ricorsivamente 'data' nei parent (fallback)
    for p in [cwd] + list(cwd.parents):
        candidates = list(p.glob('**/data'))
        for cand in candidates:
            if cand.is_dir():
                return cand.resolve()
    raise FileNotFoundError(f"Nessuna cartella 'data' trovata. Ho cercato a partire da {cwd} e suoi parent.")

data_dir = find_data_dir()
csv_files = list(data_dir.glob('*.csv'))
if not csv_files:
    raise FileNotFoundError(f'Nessun file CSV trovato nella cartella {data_dir}')
candidates = [f for f in csv_files if 'Prices_Export' in f.name]
src = candidates[0] if candidates else csv_files[0]
print(f'File sorgente usato: {src}')

# Carica il CSV (evitiamo problemi con tipi misti)
df = pd.read_csv(src, encoding='utf-8', low_memory=False)
print('Dimensione originale:', df.shape)
print('Colonne disponibili:', list(df.columns))

# Trova la colonna che probabilmente contiene il paese
possible = [c for c in df.columns if c.lower() in ('country','paese','location','countryname','country_name','nome_paese')]
if not possible:
    # fallback: cerca colonne che contengono le parole 'country' o 'paese' nel nome
    possible = [c for c in df.columns if 'country' in c.lower() or 'paese' in c.lower() or 'location' in c.lower()]
if not possible:
    raise ValueError('Non è stata trovata una colonna che possa rappresentare il paese. Controlla le colonne elencate sopra.')
country_col = possible[0]
print(f'Uso la colonna paese: {country_col}')

# Filtra righe che contengono 'yemen' (case-insensitive) nella colonna paese
mask = df[country_col].astype(str).str.contains('yemen', case=False, na=False)
yemen_df = df[mask].copy()
print('Righe trovate per Yemen:', len(yemen_df))

# Salva il CSV filtrato
out = data_dir / 'yemen_prices.csv'
yemen_df.to_csv(out, index=False, encoding='utf-8-sig')
print(f'Salvato CSV filtrato in: {out}')

# Mostra le prime righe per controllo rapido
yemen_df.head()

FileNotFoundError: Nessun file CSV trovato nella cartella data/